# 说明

> 本段代码主要处理时间和交易、展示相关的数据，旨在找出某件商品的展示和交易关系，用于做预测。

## 用户行为

> (1) 每个用户每天浏览淘宝的次数

> (2) 每个用户每小时浏览淘宝的次数

## 商品展示行为

> (3) 每件商品每天展示的次数

> (4) 每件商品每小时展示的次数

> (5) 每件商品（item）每天展示给用户（user）的次数

> (6) 每件商品每小时展示给用户的次数

> (7) 每个商品每天被购买的次数

> (8) 每个用户每天购买展示商品的次数

> 统计每件商品（item_id）每天交易次数占展示次数的比例，每两天交易次数占展示次数的比例

In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv('./data/round1_ijcai_18_train_20180301.txt', sep=' ', encoding='utf-8')
data.head()

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4016,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0


## 第一类特征：动态特征

> 这类特征跟timestamp有关，用来统计不同时间段用户行为和商品展示信息。

In [30]:
# 时间转换函数
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    dt = time.localtime(value)
    dt = time.strftime(format, dt)
    dt = datetime.datetime.strptime(dt, format)
    return dt

In [4]:
# 时间转换
data['time'] = data.context_timestamp.apply(timestamp_datetime)
data['day'] = data.time.apply(lambda x: int(x.day))
data['hour'] = data.time.apply(lambda x: int(x.hour))

In [32]:
# 每个用户每天浏览淘宝的次数
user_query_day = data.groupby(['user_id', 'day']).size().reset_index().rename(columns={0: 'user_query_day'})
data = pd.merge(data, user_query_day, 'left', on=['user_id', 'day'])
user_query_day.head()

,user_id,day,user_query_day
0,24779788309075,19,3
1,36134987234568,22,1
2,59341486148291,22,1
3,179317972644611,22,7
4,179317972644611,23,1


In [6]:
# 每个用户每小时浏览淘宝的次数
user_query_day_hour = data.groupby(['user_id', 'day', 'hour']).size().reset_index()\
                            .rename(columns={0: 'user_query_day_hour'})
data = pd.merge(data, user_query_day_hour, 'left', on=['user_id', 'day', 'hour'])
user_query_day_hour.head()

,user_id,day,hour,user_query_day_hour
0,24779788309075,19,21,1
1,24779788309075,19,22,2
2,36134987234568,22,12,1
3,59341486148291,22,19,1
4,179317972644611,22,21,7


In [7]:
# 每件商品每天展示的次数
item_display_day = data.groupby(['item_id', 'day']).size().reset_index().rename(columns={0: 'item_display_day'})
data = pd.merge(data, item_display_day, 'left', on=['item_id', 'day'])
item_display_day.head()

,item_id,day,item_display_day
0,696490723789804,18,3
1,696490723789804,19,1
2,696490723789804,20,1
3,696490723789804,21,2
4,696490723789804,22,4


In [8]:
# 每件商品每小时展示的次数
item_display_hour = data.groupby(['item_id', 'hour']).size().reset_index().rename(columns={0: 'item_display_hour'})
data = pd.merge(data, item_display_hour, 'left', on=['item_id', 'hour'])
item_display_hour.head()

,item_id,hour,item_display_hour
0,696490723789804,8,2
1,696490723789804,9,1
2,696490723789804,10,1
3,696490723789804,11,1
4,696490723789804,13,1


In [9]:
# 每件商品（item）每天展示给用户（user）的次数
user_item_query_day = data.groupby(['item_id', 'user_id', 'day']).size().reset_index()\
                            .rename(columns={0:'user_item_display_day'})
data = pd.merge(data, user_item_query_day, on=['item_id', 'user_id', 'day'], how='left')
user_item_query_day.head()

,item_id,user_id,day,user_item_display_day
0,696490723789804,592559664920555719,20,1
1,696490723789804,1339215130517887336,18,1
2,696490723789804,3163069223182186215,22,1
3,696490723789804,3501198089440436410,21,1
4,696490723789804,4023694609561949813,22,1


In [10]:
# 每件商品每小时展示给用户的次数
user_item_query_hour = data.groupby(['item_id', 'user_id', 'day', 'hour']).size().reset_index()\
                            .rename(columns={0:'user_item_display_hour'})
data = pd.merge(data, user_item_query_hour, on=['item_id', 'user_id', 'day', 'hour'], how='left')
user_item_query_hour.head()

,item_id,user_id,day,hour,user_item_display_hour
0,696490723789804,592559664920555719,20,8,1
1,696490723789804,1339215130517887336,18,10,1
2,696490723789804,3163069223182186215,22,21,1
3,696490723789804,3501198089440436410,21,9,1
4,696490723789804,4023694609561949813,22,19,1


In [11]:
# 每个商品每天被购买的次数
item_buy_day = data[['item_id', 'day', 'is_trade']].groupby(['item_id', 'day']).sum().reset_index()\
                                                    .rename(columns={'is_trade': 'item_buy_day'})
data = pd.merge(data, item_buy_day, on=['item_id', 'day'], how='left')
item_buy_day.sort_values(by='item_buy_day', ascending=False).head()

,item_id,day,item_buy_day
23512,5410526681843914464,18,26
13081,2986722878397696833,18,25
13083,2986722878397696833,20,21
9032,2079092401639430105,20,21
9034,2079092401639430105,22,19


In [12]:
# 每个用户每天购买的次数
user_buy_day = data[['user_id', 'day', 'is_trade']].groupby(['user_id', 'day']).sum().reset_index()\
                                                    .rename(columns={'is_trade': 'user_buy_day'})
data = pd.merge(data, user_buy_day, on=['user_id', 'day'], how='left')
user_buy_day.sort_values(by='user_buy_day', ascending=False).head()

,user_id,day,user_buy_day
49162,1965358337508268958,22,3
181754,7302738499556980530,20,3
75851,3056263399386680541,18,3
97177,3919489572177177656,21,3
206425,8290365491518637486,18,3


In [13]:
# 每个用户每天购买展示商品的次数
user_item_buy_day = data[['item_id', 'user_id', 'day', 'is_trade']].groupby(['item_id', 'user_id', 'day'])\
                                                                    .sum().reset_index()\
                                                                    .rename(columns={'is_trade': 'user_item_buy_day'})
data = pd.merge(data, user_item_buy_day, on=['item_id', 'user_id', 'day'], how='left')
user_item_buy_day.head()

,item_id,user_id,day,user_item_buy_day
0,696490723789804,592559664920555719,20,0
1,696490723789804,1339215130517887336,18,0
2,696490723789804,3163069223182186215,22,0
3,696490723789804,3501198089440436410,21,1
4,696490723789804,4023694609561949813,22,0


In [14]:
data.head()

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,hour,user_queryy_day,user_query_day_hour,item_display_day,item_display_hour,user_item_display_day,user_item_display_hour,item_buy_day,user_buy_day,user_item_buy_day
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,10,3,3,6,2,1,1,0,0,0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,12,8,5,6,1,1,1,0,0,0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,3,2,2,6,1,1,1,0,0,0
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,6,1,1,6,1,1,1,0,0,0
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,19,2,2,6,2,1,1,0,0,0


## 第二类特征

> 第二类特征是统计每一天中，每一个特征的取值在is_trade取0或1的个数占这个特征所有取值在is_trade取值为0或1总数的比例。

> 这类特征有：
            item_id, item_property_list, item_brand_id, item_city_id, user_gender_id, user_age_level,
            user_occupation_id, user_star_level, context_page_id, shop_star_level

>

>

>

In [15]:
def feature_group(data, feature, is_trade):
    if (is_trade != 1 and is_trade != 0):
        return None
    else:
        data = data[[feature, 'is_trade']]
        data_feature = data[data['is_trade'] == is_trade]
        data_feature = data_feature.reset_index(drop=True)
        data_feature_grouped = data_feature.groupby([feature]).count()
        data_feature_grouped = data_feature_grouped.reset_index()
        data_feature_grouped = data_feature_grouped.sort_values(['is_trade'], ascending=False)   
    return data_feature_grouped

In [16]:
def feature_log_cal(data, feature, is_trade):
    if (is_trade != 1 and is_trade != 0):
        return None
    else:
        data_feature_grouped = feature_group(data, feature, is_trade)
        new_feature = feature + '_' + str(is_trade)
        data_feature_grouped[new_feature] = np.log(data_feature_grouped['is_trade'] / len(data_feature_grouped))
#         # 0-1标准化
#         data_feature_grouped[new_feature] =  (data_feature_grouped['is_trade'] - data_feature_grouped['is_trade'].min())/\
#                                         (data_feature_grouped['is_trade'].max() - data_feature_grouped['is_trade'].min())
    return data_feature_grouped[[feature, new_feature]]

In [17]:
raw_features = ['item_id', 'item_category_list', 'item_property_list', 'item_brand_id', 'item_city_id', 
                'user_id', 'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level',
               'shop_id', 'context_page_id', 'predict_category_property']

In [18]:
# data_list中存储这每天，每个交易类别的数据
data_list = []
for day in range(18, 25, 1):
    for i in range(2):
        data_day = data[data.day == day]
        data_day_trade = data_day[data_day.is_trade == i]
        data_list.append(data_day_trade)

In [19]:
len(data)

478138

In [20]:
new_data_list = []
for dl in data_list:
    day = dl.day.unique()[0]
    is_trade_flag = dl.is_trade.unique()[0]
    for feature in raw_features:
        flc = feature_log_cal(dl, feature, is_trade_flag)
        dl = pd.merge(dl, flc, on=feature, how='left')
    new_data_list.append(dl)

In [21]:
whole_data = pd.DataFrame()
for i in new_data_list:
    whole_data = pd.concat([whole_data, i], axis=0)

In [22]:
len(whole_data)

478138

In [23]:
whole_data.head()

,context_id,context_page_id,context_page_id_0,context_page_id_1,context_timestamp,day,hour,instance_id,is_trade,item_brand_id,...,user_item_display_day,user_item_display_hour,user_occupation_id,user_occupation_id_0,user_occupation_id_1,user_query_day_hour,user_queryy_day,user_star_level,user_star_level_0,user_star_level_1
0,282924576738839389,4006,4.788741,NaN,1537236544,18,10,108641074714126964,0,1975590437749032870,...,1,1,2005,9.172223,NaN,3,3,3003,6.915558,NaN
1,4007979028023783431,4001,7.669052,NaN,1537243232,18,12,5754713551599725161,0,1975590437749032870,...,1,1,2005,9.172223,NaN,5,8,3006,7.003368,NaN
2,4884875192608989870,4001,7.669052,NaN,1537211052,18,3,842679481291040981,0,1975590437749032870,...,1,1,2005,9.172223,NaN,2,2,3004,6.737165,NaN
3,840119421106178602,4016,3.261935,NaN,1537222670,18,6,937088850059189027,0,1975590437749032870,...,1,1,2005,9.172223,NaN,1,1,3006,7.003368,NaN
4,1736769971710354684,4001,7.669052,NaN,1537271320,18,19,7975697065017708072,0,1975590437749032870,...,1,1,2005,9.172223,NaN,2,2,3001,5.531411,NaN


In [25]:
tmp_features = [f + '_0' for f in raw_features] + [f + '_1' for f in raw_features]
for f in tmp_features:
    whole_data[f].fillna(whole_data[f].min(), inplace=True)

In [26]:
whole_data.head()

,context_id,context_page_id,context_page_id_0,context_page_id_1,context_timestamp,day,hour,instance_id,is_trade,item_brand_id,...,user_item_display_day,user_item_display_hour,user_occupation_id,user_occupation_id_0,user_occupation_id_1,user_query_day_hour,user_queryy_day,user_star_level,user_star_level_0,user_star_level_1
0,282924576738839389,4006,4.788741,-2.995732,1537236544,18,10,108641074714126964,0,1975590437749032870,...,1,1,2005,9.172223,-1.609438,3,3,3003,6.915558,-2.484907
1,4007979028023783431,4001,7.669052,-2.995732,1537243232,18,12,5754713551599725161,0,1975590437749032870,...,1,1,2005,9.172223,-1.609438,5,8,3006,7.003368,-2.484907
2,4884875192608989870,4001,7.669052,-2.995732,1537211052,18,3,842679481291040981,0,1975590437749032870,...,1,1,2005,9.172223,-1.609438,2,2,3004,6.737165,-2.484907
3,840119421106178602,4016,3.261935,-2.995732,1537222670,18,6,937088850059189027,0,1975590437749032870,...,1,1,2005,9.172223,-1.609438,1,1,3006,7.003368,-2.484907
4,1736769971710354684,4001,7.669052,-2.995732,1537271320,18,19,7975697065017708072,0,1975590437749032870,...,1,1,2005,9.172223,-1.609438,2,2,3001,5.531411,-2.484907


In [28]:
columns = whole_data.columns
columns

Index(['context_id', 'context_page_id', 'context_page_id_0',
       'context_page_id_1', 'context_timestamp', 'day', 'hour', 'instance_id',
       'is_trade', 'item_brand_id', 'item_brand_id_0', 'item_brand_id_1',
       'item_buy_day', 'item_category_list', 'item_category_list_0',
       'item_category_list_1', 'item_city_id', 'item_city_id_0',
       'item_city_id_1', 'item_collected_level', 'item_display_day',
       'item_display_hour', 'item_id', 'item_id_0', 'item_id_1',
       'item_price_level', 'item_property_list', 'item_property_list_0',
       'item_property_list_1', 'item_pv_level', 'item_sales_level',
       'predict_category_property', 'predict_category_property_0',
       'predict_category_property_1', 'shop_id', 'shop_id_0', 'shop_id_1',
       'shop_review_num_level', 'shop_review_positive_rate',
       'shop_score_delivery', 'shop_score_description', 'shop_score_service',
       'shop_star_level', 'time', 'user_age_level', 'user_age_level_0',
       'user_age_level_1

In [29]:
len(columns)

65

# 由于是用历史数据来预测未来的数据，因此需要进行数据平移

> 日期：18至25号(25号为测试数据，目标变量不知，需要预测。)

> 使用18、19号的数据预测20号，19、20号的数据预测21号的，以此类推

> 需要平移的特征：
            >>   第一类特征：
                   
                   user_queryy_day, user_query_day_hour, item_display_day, item_display_hour, 
                   user_item_display_day, user_item_display_hour, item_buy_day, user_buy_day, user_item_buy_day

                
             >>   第二类特征：
                
                item_id_0, item_id_1, item_property_list_0, item_property_list1, item_brand_id_0, item_brand_id_1,
                item_city_id_0, item_city_id_1, user_gender_id_0, user_gender_id_1, user_age_level_0, 
                user_age_level_1,
                user_occupation_id_0, user_occupation_id_1, user_star_level_0, user_star_level_1, 
                context_page_id_0, context_page_id_1, shop_id_0, shop_id_1, shop_star_level_0, shop_star_level_1,
                
> 固定特征：
          
          item_id, item_price_level, item_sales_level, item_collected_level, item_pv_level,
          user_id, shop_review_num_level, shop_review_positive_rate, shop_score_service, shop_score_delivery, 
          shop_score_description


In [33]:
fixed_features = ['item_id', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level',
                'user_id', 'shop_id', 'shop_review_num_level', 'shop_review_positive_rate', 'shop_score_service', 
                 'shop_score_delivery', 'shop_score_description']
translation_features = ['user_query_day', 'user_query_day_hour', 'item_display_day', 'item_display_hour', 
                'user_item_display_day', 'user_item_display_hour', 'item_buy_day', 'user_buy_day', 'user_item_buy_day',
                'item_id_0', 'item_id_1', 'item_property_list_0', 'item_property_list_1', 'item_brand_id_0', 
                'item_brand_id_1', 'item_city_id_0', 'item_city_id_1', 'user_gender_id_0', 'user_gender_id_1', 
                'user_age_level_0', 'user_age_level_1','user_occupation_id_0', 'user_occupation_id_1', 
                'user_star_level_0', 'user_star_level_1', 'context_page_id_0', 'context_page_id_1', 'shop_id_0', 
                'shop_id_1', 'shop_star_level_0', 'shop_star_level_1']

In [ ]:
data_list = []
for day in range(18, 25, 1):
    data_day = data[data.day == day] 
    data_day[fixed_features].append()

# 测试集